In [41]:
import json

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
from shapely.geometry import LineString, Point

from railroad import (CustomJSONEncoder, custom_json_decoder, load_graph,
                      plot_node_graph, save_graph, test_road_graph)
from utils import osmnx_to_geopandas, print_graph_info, compute_length, find_closest_point_id, get_interest_matrixes_from_graph_with_scores

In [42]:
with open("../data/standard_variables.json", "r") as f:
    standard_variables_data = json.load(f)


xlim = (standard_variables_data['map_limits']['west'],
        standard_variables_data['map_limits']['east'])
ylim = (standard_variables_data['map_limits']['south'],
        standard_variables_data['map_limits']['north'])
print(xlim, ylim)

standard_variables_data

(-22.083955, -21.640345) (64.033992, 64.188968)


{'map_limits': {'north': 64.188968,
  'south': 64.033992,
  'east': -21.640345,
  'west': -22.083955},
 'grid_size': {'unit': 'meters', 'value': 100},
 'standard_variables': {'cutoff_radius': {'unit': 'meters', 'value': 400}}}

In [43]:
filename = 'final_railroads_2030'

In [44]:
G = load_graph(f'../data/{filename}.json')

In [45]:
len(G.nodes), len(G.edges)

(89, 94)

In [46]:
G.nodes[1848219055]

{'street_count': 3,
 'geometry': <POINT (-21.81 64.086)>,
 'year': 2025,
 'is_station': True,
 'if_final_stop_of_any_kind': True}

In [47]:
results_df = pd.read_csv("../data/lattice_normalized_score.csv")
display(results_df)

,point_index,latitude,longitude,location,total_normalized_score,Tourism_score,GymsSports_score,Commercial Dist._score,ParksStreetRecreationReligious._score,Pools_score,Adm. Districts_score,EduLibHealthcare_score,Trans. Hub_score,nearby_pois
0,0,64.033992,-22.083955,"(-22.083955, 64.033992)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Empty GeoDataFrame\nColumns: [Name, Latitude, ..."
1,1,64.033992,-22.083057,"(-22.083056684715878, 64.033992)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Empty GeoDataFrame\nColumns: [Name, Latitude, ..."
2,2,64.033992,-22.082158,"(-22.08215836943176, 64.033992)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Empty GeoDataFrame\nColumns: [Name, Latitude, ..."
3,3,64.033992,-22.081260,"(-22.08126005414764, 64.033992)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Empty GeoDataFrame\nColumns: [Name, Latitude, ..."
4,4,64.033992,-22.080362,"(-22.08036173886352, 64.033992)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Empty GeoDataFrame\nColumns: [Name, Latitude, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195619,195619,64.188920,-21.644679,"(-21.64467882606555, 64.18892008786918)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Empty GeoDataFrame\nColumns: [Name, Latitude, ..."
195620,195620,64.188920,-21.643781,"(-21.643780510781433, 64.18892008786918)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Empty GeoDataFrame\nColumns: [Name, Latitude, ..."
195621,195621,64.188920,-21.642882,"(-21.642882195497315, 64.18892008786918)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Empty GeoDataFrame\nColumns: [Name, Latitude, ..."
195622,195622,64.188920,-21.641984,"(-21.641983880213193, 64.18892008786918)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Empty GeoDataFrame\nColumns: [Name, Latitude, ..."


In [48]:
x_lattice_array = sorted(results_df['longitude'].unique())
y_lattice_array = sorted(results_df['latitude'].unique())

In [49]:
test_x = -21.85  # Example longitude
test_y = 64.034   # Example latitude

closest_id = find_closest_point_id(
    test_x, test_y, x_lattice_array, y_lattice_array)
print(f"Closest point ID for ({test_x}, {test_y}): {closest_id}")
display(results_df.loc[closest_id])

Closest point ID for (-21.85, 64.034): 260


point_index                                                                            260
latitude                                                                         64.033992
longitude                                                                       -21.850393
location                                                  (-21.850393026128923, 64.033992)
total_normalized_score                                                                 0.0
Tourism_score                                                                          0.0
GymsSports_score                                                                       0.0
Commercial Dist._score                                                                 0.0
ParksStreetRecreationReligious._score                                                  0.0
Pools_score                                                                            0.0
Adm. Districts_score                                                                   0.0

In [50]:
score_columns = [column for column in results_df.columns if '_score' in column]
print(score_columns)

['total_normalized_score', 'Tourism_score', 'GymsSports_score', 'Commercial Dist._score', 'ParksStreetRecreationReligious._score', 'Pools_score', 'Adm. Districts_score', 'EduLibHealthcare_score', 'Trans. Hub_score']


In [51]:
for node_id in G.nodes:
    x = G.nodes[node_id]['geometry'].x
    y = G.nodes[node_id]['geometry'].y
    print(f"Node ID: {node_id}, x: {x}, y: {y}")
    closest_id = find_closest_point_id(x, y, x_lattice_array, y_lattice_array)
    data = results_df.loc[closest_id][score_columns]
    print(data)
    G.nodes[node_id].update(data.to_dict())

Node ID: 1848219055, x: -21.8096127, y: 64.0856269
total_normalized_score                   0.0
Tourism_score                            0.0
GymsSports_score                         0.0
Commercial Dist._score                   0.0
ParksStreetRecreationReligious._score    0.0
Pools_score                              0.0
Adm. Districts_score                     0.0
EduLibHealthcare_score                   0.0
Trans. Hub_score                         0.0
Name: 65019, dtype: object
Node ID: 602729079, x: -21.8438709, y: 64.0897624
total_normalized_score                   0.021155
Tourism_score                                 0.0
GymsSports_score                              0.0
Commercial Dist._score                   0.021155
ParksStreetRecreationReligious._score         0.0
Pools_score                                   0.0
Adm. Districts_score                          0.0
EduLibHealthcare_score                        0.0
Trans. Hub_score                              0.0
Name: 70415, dtyp

In [52]:
save_graph(G, f'../data/{filename}_with_scores.json')

In [53]:
common_interest_df, tourist_df, common_interest_matrix, tourist_matrix = get_interest_matrixes_from_graph_with_scores(
    G)
# Display matrices
print("Common Interest Matrix:")
display(common_interest_df)
print(f"Common Matrix max: {common_interest_matrix.max()}")

print("\nTourist Matrix:")
display(tourist_df)
print(f"Tourist Matrix max: {tourist_matrix.max()}")

Common Interest Matrix:


,1848219055,602729079,111465687,2320789566,2320771865,2328470640,1889922204,1896201758,470330027,384966546,...,12339218115,12339218116,12339218117,12339218118,12339218119,12339218120,12339218121,12339218122,12339218123,12339218124
1848219055,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
602729079,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
111465687,0.0,0.0,0.0,0.314787,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2320789566,0.0,0.0,0.0,0.000000,0.314787,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2320771865,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12339218120,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
12339218121,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
12339218122,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.149854,0.000000
12339218123,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.085466


Common Matrix max: 0.5783795218954939

Tourist Matrix:


,1848219055,602729079,111465687,2320789566,2320771865,2328470640,1889922204,1896201758,470330027,384966546,...,12339218115,12339218116,12339218117,12339218118,12339218119,12339218120,12339218121,12339218122,12339218123,12339218124
1848219055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
602729079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
111465687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2320789566,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2320771865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12339218120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
12339218121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
12339218122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003043,0.0
12339218123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


Tourist Matrix max: 0.04289376607580315
